In [9]:

from transformers import pipeline

#set device
device = 0




audio_file=r"C:\Users\gteja\Downloads\medical_audio_file.mp3"

pipe = pipeline(model='Bhaveen/500medimix', device=device,return_timestamps=True)
output = pipe(audio_file)

Device set to use cuda:0
C:\Users\gteja\AppData\Roaming\Python\Python312\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [10]:
print(output['text'])

 Yes. Four is what? Costectomy, left rear assist. All right, let's do it. Thank you. What is the history? Line base negative. Levered is fine. Spleen is fine. Gall butter. No, sorry, we're going to go to FOSSA. can we call it Dectomy, pancreas is fine, thumbtack is fine, mobile loops are fine. Just add them. Add three cinema left re-assist. I'll bring it up. A very simple system. It's 15HU. Okay. Oh, before we do that. Rheumatism, erinormal cell monosidicization. IVC endomarcoval, no brainer. Opethyriacine normal. Left kidney normal, but at assist. comparison stable. Emily Sengit, chest to views. Zero, orange. Raoul, P.A.5. Left ankle. Right knee, 0-1-2. foot zero one two All fractures neck of third worth of fifth no terms of bonds Right needs a one one.


In [1]:
from flask import Flask, render_template, request, jsonify
import os
from flask_cors import CORS
import openai
import datetime

import firebase_admin
from firebase_admin import db,storage,credentials
import datetime
import sys
from Firebase.pdf import pdf_conv
from Firebase.firebase_stuff import create_doctor_account, upload_pdf_to_bucket, download_prescription, login_firebase
#from Firebase.pharmacist import pharmacist
from Firebase.register_firebase import registration
from Firebase.text_formatting import formatting
from transformers import pipeline

device = 0
pipe = pipeline(model='Bhaveen/500medimix', device=device,return_timestamps=True)


cred = credentials.Certificate('Firebase/credentials_new.json')
firebase_admin.initialize_app(
    cred,
    {'databaseURL' : 'https://dr-writely-default-rtdb.asia-southeast1.firebasedatabase.app/','storageBucket': 'dr-writely.appspot.com'}
)

bucket = storage.bucket()

Device set to use cuda:0


In [43]:
# patient name -> patient phno
# date
# diagnosis
# medication
 
def get_doctor(doc_name,db):
    result = {}
    refD = db.reference(f'/User/Doctor/{doc_name}')
    refC = db.reference(f'/User/Patient')
    ch = refD.get()
    patients = [i for i in ch.keys()]
    chP = refC.get()
    for i,j in chP.items():
        if i in patients:
            for k in j:
                temp_dict = {'patient_phno': '', 'Date': '', 'Diagnosis': '', 'medication': ''}
                temp_dict['Date']=(j[k]['date'])
                temp_dict['Diagnosis']=(j[k]['diagnosis'])
                temp_dict['medication']=(j[k]['medication'])
                temp_dict['patient_phno']=(i)
                result[k] = temp_dict
    print(result)


    

In [44]:
get_doctor('Tejas',db)

{'202502012208': {'patient_phno': '9019942460', 'Date': '2025-02-01', 'Diagnosis': ' Clamydia (Chlamydia) infection  ', 'medication': [' azithromycin', ' single dose; doxycycline', ' 100 mg twice daily for 7 days  ']}}


In [87]:
def get_patient(phno,db):
    result = {}
    refP = db.reference(f'/User/Patient/{phno}')
    chP = refP.get()
    refD = db.reference(f'/User/Doctor')
    chD = refD.get()
    appts = [i for i in chP.keys()]
    print(appts)
    for i,j in chD.items():
        temp_dict = {'date':'','diagnosis':'','medication':[]}
        if phno in list(j.keys()):
            for k in appts:
                print(j[phno][k])
                if k in j[phno].keys():
                    temp_dict['date'] = j[phno][k]['date']
                    temp_dict['date'] = j[phno][k]['date']
                    temp_dict['diagnosis'] = j[phno][k]['diagnosis']
                    temp_dict['medication'] = j[phno][k]['medication']
                    result[i].append(temp_dict)
            



In [88]:
get_patient('9019942460',db)

['202502012208']
{'date': '2025-02-01', 'transcript': ' Clamydia is a bacterial infection. Clamydia trachomatis causing genital pain, discharge or no symptoms. Treatment is azithromycin, single dose or doxicycline, 100 mg twice daily for 7 days. Partial treatment is advised.'}
{}


In [103]:
#ref1 = db.reference(f'/User/Doctor/{doctor}/{patient}')
patient = '9019942460'
ref2 = db.reference(f"/User/Patient/{patient}")
refC = db.reference(f'/Creds')
ch = refC.get()
#print(ch.keys())
user = [j for i,j in ch.items() if i in [patient]]
print(user[0]['name'])
# ref3 = db.reference(f"/Pharm/{user}")
# chx = ref3.get('name')
# print(chx)

dict_keys(['9019942460', '9123456766', '9123456789', '9875684832'])
Tejas


In [36]:
def get_pharmacist(db):
    result = {}
    refPharm = db.reference('/Pharm')
    ch = refPharm.get()
    for i,j in ch.items():
        result[i] = j
    return result

In [37]:
print(get_pharmacist(db))

{'Tejas': {'Phone Number': '9019942460', 'date': '2025-02-01', 'diagnosis': ' viral fever  ', 'medication': [' paracetamol 500-650 mg  ']}}


In [30]:
def get_doctor(doc_ph,db):
    result = {}
    refCred = db.reference(f'/Creds')
    chx = refCred.get()
    doc = [j for i,j in chx.items() if i in [doc_ph]]
    #print(doc)
    doc_name = doc[0]['name']
    refD = db.reference(f'/User/Doctor/{doc_name}')
    refC = db.reference(f'/User/Patient')
    ch = refD.get()
    patients = [i for i in ch.keys()]
    chP = refC.get()
    for i,j in chP.items():
        if i in patients:
            for k in j:
                temp_dict = {'patient_phno': '', 'Date': '', 'Diagnosis': '', 'medication': ''}
                temp_dict['date']=(j[k]['date'])
                temp_dict['diagnosis']=(j[k]['diagnosis'])
                temp_dict['medication']=(j[k]['medication'])
                temp_dict['patient_phno']=(i)
                result[k].append(temp_dict)
    return result

In [31]:
print(get_doctor('9123456766',db))

KeyError: '202502012347'

In [66]:
def get_patient(phno,db):
    
    refP = db.reference(f'/User/Patient/{phno}')
    chP = refP.get()
    refD = db.reference(f'/User/Doctor')
    chD = refD.get()
    refCred = db.reference(f'/Creds')
    chx = refCred.get()
    #print(chx.items())
    doc = [j for i,j in chx.items() if i in [phno]]
    result = {doc[0]['name']:[]}    
    appts = [i for i in chP.keys()]
    #print(appts)
    for i,j in chD.items():
        temp_dict = {'date':'','diagnosis':'','medication':[],'Doc_name':''}
        if phno in list(j.keys()):
            for k in appts:
                #print(i,j)
                if k in j[phno].keys():
                    
                    temp_dict['date'] = j[phno][k]['date']
                    temp_dict['diagnosis'] = j[phno][k]['diagnosis']
                    temp_dict['medication'] = j[phno][k]['medication']
                    temp_dict['appt'] = k
                    temp_dict['Doc_name'] = i
                    result[doc[0]['name']].append(temp_dict)
    return result

In [67]:
print(get_patient('9019942460',db))

{'Tejas': [{'date': '2025-02-02', 'diagnosis': '  NA  ', 'medication': ['  Azithromycin 100mg 3 times a day for 3 days', ' fever medication 650mg  '], 'Doc_name': 'Jishnu Phani Perisetti', 'appt': '202502020527'}, {'date': '2025-02-01', 'diagnosis': ' viral fever  ', 'medication': [' paracetamol 500-650 mg  '], 'Doc_name': 'Tejas', 'appt': '202502012350'}, {'date': '2025-02-01', 'diagnosis': ' viral fever  ', 'medication': [' paracetamol 500-650 mg  '], 'Doc_name': 'Tejas', 'appt': '202502012350'}]}


In [48]:
def download_prescription(appt_id,bucket):
    blobs = bucket.list_blobs(prefix=f'{appt_id}/')
    name = ""
    for i in blobs:
        name = (i.name)
    dest_nm = name.split('/')[-1]
    bucket.blob(name).download_to_filename(f'{dest_nm}')

In [49]:
download_prescription(202502012350,bucket)

In [70]:
def get_doctor(doc_ph, db):
    try:
        result = {}
        # Get doctor credentials
        refCred = db.reference('/Creds')
        chx = refCred.get()
        doc = [j for i,j in chx.items() if i in [doc_ph]]
        if not doc:
            return {"error": "Doctor not found"}
        
        doc_name = doc[0]['name']
        
        # Get doctor's patients
        refD = db.reference(f'/User/Doctor/{doc_name}')
        refC = db.reference('/User/Patient')
        
        ch = refD.get()
        if not ch:
            return {"error": "No patient records found"}
            
        patients = list(ch.keys())
        chP = refC.get()

        # Process patient records
        for patient_ph, records in chP.items():
            if patient_ph in patients:
                for record_id, record in records.items():
                    
                    result[record_id] = {
                        'patient_phno': patient_ph.strip(),
                        'date': record.get('date', '').strip(),
                        'diagnosis': record.get('diagnosis', '').strip(),
                        'medication': record.get('medication', []),
                        'doctor_name': doc_name,
                        'appt':record_id
                    }
        
        return {
            "status": "success",
            "data": result,
            "doctor_name": doc_name
        }
        
    except Exception as e:
        return {
            "status": "error",
            "message": str(e)
        }

In [71]:
get_doctor('9972710948',db)

202502012347
202502012350
202502020527


{'status': 'success',
 'data': {'202502012347': {'patient_phno': '9019942460',
   'date': '2025-02-01',
   'diagnosis': 'Viral fever',
   'medication': ['\tParacetamol (500 to 650 mg)', ' Ibuprofen if needed  '],
   'doctor_name': 'Jishnu Phani Perisetti'},
  '202502012350': {'patient_phno': '9019942460',
   'date': '2025-02-01',
   'diagnosis': 'viral fever',
   'medication': [' paracetamol 500-650 mg  '],
   'doctor_name': 'Jishnu Phani Perisetti'},
  '202502020527': {'patient_phno': '9019942460',
   'date': '2025-02-02',
   'diagnosis': 'NA',
   'medication': ['  Azithromycin 100mg 3 times a day for 3 days',
    ' fever medication 650mg  '],
   'doctor_name': 'Jishnu Phani Perisetti'}},
 'doctor_name': 'Jishnu Phani Perisetti'}